## Astrophysics background

It is very common in Astrophysics to work with sky pixels. The sky is tassellated in patches with specific properties and a sky map is then a collection of intensity values for each pixel. The most common pixelization used in Cosmology is [HEALPix](http://healpix.jpl.nasa.gov).

Measurements from telescopes are then represented as an array of pixels that encode the pointing of the instrument at each timestamp and the measurement output.

## Sample timeline

In [1]:
import pandas as pd
import numba
import numpy as np

For simplicity let's assume we have a sky with 50K pixels:

In [2]:
NPIX = 50000

And we have 50 million measurement from our instrument:

In [3]:
NTIME = int(50 * 1e6)

The pointing of our instrument is an array of pixels, random in our sample case:

In [4]:
pixels = np.random.randint(0, NPIX-1, NTIME)

Our data are also random:

In [5]:
timeline = np.random.randn(NTIME)

## Create a map of the sky with pandas

One of the most common operations is to sum all of our measurements in a sky map, so the value of each pixel in our sky map will be the sum of each individual measurement.
The easiest way is to use the `groupby` operation in `pandas`:

In [6]:
timeline_pandas = pd.Series(timeline, index=pixels)

In [7]:
timeline_pandas.head()

39614    1.410522
29270   -0.982586
24793    2.597090
40142   -0.279812
11449    1.334712
dtype: float64

In [8]:
%time m = timeline_pandas.groupby(level=0).sum()

CPU times: user 724 ms, sys: 101 ms, total: 824 ms
Wall time: 828 ms


## Create a map of the sky with numba

We would like to improve the performance of this operation using `numba`, which allows to produce automatically C-speed compiled code from pure python functions.

First we need to develop a pure python version of the code, test it, and then have `numba` optimize it:

In [9]:
def groupby_python(index, value, output):
    for i in range(index.shape[0]):
        output[index[i]] += value[i]

In [10]:
m_python = np.zeros_like(m)

In [11]:
%time groupby_python(pixels, timeline, m_python)

CPU times: user 22.9 s, sys: 2 ms, total: 22.9 s
Wall time: 23 s


In [12]:
np.testing.assert_allclose(m_python, m)

Pure Python is slower than the `pandas` version implemented in `cython`.

### Optimize the function with numba.jit

`numba.jit` gets an input function and creates an compiled version with does not depend on slow Python calls, this is enforced by `nopython=True`, `numba` would throw an error if it would not be possible to run in `nopython` mode.

In [13]:
groupby_numba = numba.jit(groupby_python, nopython=True)

In [14]:
m_numba = np.zeros_like(m)

In [15]:
%time groupby_numba(pixels, timeline, m_numba)

CPU times: user 412 ms, sys: 133 ms, total: 545 ms
Wall time: 429 ms


In [16]:
m_numba = np.zeros_like(m)

In [17]:
%time groupby_numba(pixels, timeline, m_numba)

CPU times: user 75.2 ms, sys: 0 ns, total: 75.2 ms
Wall time: 75.7 ms


In [18]:
np.testing.assert_allclose(m_numba, m)

Performance improvement is about 50x compared to Python and up to 10x compared to Pandas, pretty good!

## Use numba.jit as a decorator

The exact same result is obtained if we use `numba.jit` as a decorator:

In [19]:
@numba.jit(nopython=True)
def groupby_numba(index, value, output):
    for i in range(index.shape[0]):
        output[index[i]] += value[i]

## Example of numba in a Python library

* https://github.com/galsci/pysm/blob/main/pysm3/models/power_law.py#L72-L100
* https://github.com/galsci/pysm/blob/main/pysm3/utils/__init__.py#L131-L158
    